# Yelp API - Lab


## Introduction 

Now that we've seen how the Yelp API works and some basic Folium visualizations, it's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [9]:
#Your code here
api_key = 's-YYryUeiCblWtWQJHzyhkz6hAz6Enl_Ipc1wOFIm-ZvDXw8zJnhAfqzWqh6JMrKor00z7g9OZZfUNtL9g4fV7nVamff7YFUmEflUR9DuLgCgi0WjfsvD5dB8nPcXHYx'

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [10]:
# Your code here; use a function or loop to retrieve all the results from your original request

In [11]:
import requests
term = 'Pizza'
location = 'Vienna VA'
SEARCH_LIMIT = 10

url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT
            }
response = requests.get(url, headers=headers, params=url_params)
print(response)
print(type(response.text))
print(response.text[:1000])

<Response [200]>
<class 'str'>
{"businesses": [{"id": "Bl9BKz_ZhhQ-Jui8wVwagw", "alias": "church-street-pizzeria-vienna", "name": "Church Street Pizzeria", "image_url": "https://s3-media1.fl.yelpcdn.com/bphoto/51U3_klXkqr_6QMpKeB4BQ/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/church-street-pizzeria-vienna?adjust_creative=LVcYNLGkb44VfIeQCK9tDQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=LVcYNLGkb44VfIeQCK9tDQ", "review_count": 261, "categories": [{"alias": "pizza", "title": "Pizza"}], "rating": 4.0, "coordinates": {"latitude": 38.9020047, "longitude": -77.2669418}, "transactions": [], "price": "$", "location": {"address1": "115 Church St", "address2": "", "address3": "", "city": "Vienna", "zip_code": "22180", "country": "US", "state": "VA", "display_address": ["115 Church St", "Vienna, VA 22180"]}, "phone": "+17039381155", "display_phone": "(703) 938-1155", "distance": 466.90471561225036}, {"id": "rnzimh0SVH3AeG6n5j3ZWw", "alias": "crust-pizze

In [12]:
response.json().keys() 

dict_keys(['businesses', 'total', 'region'])

In [14]:
for key in response.json().keys():
    print(key)
    value = response.json()[key] 
    print(type(value)) 
    print('\n\n')


businesses
<class 'list'>



total
<class 'int'>



region
<class 'dict'>





In [15]:
#Your code here
response.json()['businesses'][:2]

[{'id': 'Bl9BKz_ZhhQ-Jui8wVwagw',
  'alias': 'church-street-pizzeria-vienna',
  'name': 'Church Street Pizzeria',
  'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/51U3_klXkqr_6QMpKeB4BQ/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/church-street-pizzeria-vienna?adjust_creative=LVcYNLGkb44VfIeQCK9tDQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=LVcYNLGkb44VfIeQCK9tDQ',
  'review_count': 261,
  'categories': [{'alias': 'pizza', 'title': 'Pizza'}],
  'rating': 4.0,
  'coordinates': {'latitude': 38.9020047, 'longitude': -77.2669418},
  'transactions': [],
  'price': '$',
  'location': {'address1': '115 Church St',
   'address2': '',
   'address3': '',
   'city': 'Vienna',
   'zip_code': '22180',
   'country': 'US',
   'state': 'VA',
   'display_address': ['115 Church St', 'Vienna, VA 22180']},
  'phone': '+17039381155',
  'display_phone': '(703) 938-1155',
  'distance': 466.90471561225036},
 {'id': 'rnzimh0SVH3AeG6n5j3ZWw',
  'alias': 'cru

In [16]:
response.json()['total']

139

In [17]:
response.json()['region']

{'center': {'longitude': -77.26221084594727, 'latitude': 38.900782943812246}}

## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [18]:
#Your code here
import pandas as pd

df = pd.DataFrame.from_dict(response.json()['businesses'])
print(len(df)) #Print how many rows
print(df.columns) #Print column names
df.head() #Previews the first five rows. 
#You could also write df.head(10) to preview 10 rows or df.tail() to see the bottom

10
Index(['alias', 'categories', 'coordinates', 'display_phone', 'distance', 'id',
       'image_url', 'is_closed', 'location', 'name', 'phone', 'price',
       'rating', 'review_count', 'transactions', 'url'],
      dtype='object')


,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,church-street-pizzeria-vienna,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 38.9020047, 'longitude': -77.2669...",(703) 938-1155,466.904716,Bl9BKz_ZhhQ-Jui8wVwagw,https://s3-media1.fl.yelpcdn.com/bphoto/51U3_k...,False,"{'address1': '115 Church St', 'address2': '', ...",Church Street Pizzeria,+17039381155,$,4.0,261,[],https://www.yelp.com/biz/church-street-pizzeri...
1,crust-pizzeria-napoletana-vienna-2,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 38.91509, 'longitude': -77.23666}",(703) 663-8233,2746.700169,rnzimh0SVH3AeG6n5j3ZWw,https://s3-media2.fl.yelpcdn.com/bphoto/f0RGJy...,False,"{'address1': '8415 Old Courthouse Rd', 'addres...",Crust Pizzeria Napoletana,+17036638233,$$,4.0,364,"[delivery, pickup]",https://www.yelp.com/biz/crust-pizzeria-napole...
2,pazzo-pomodoro-vienna-2,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 38.906699123933, 'longitude': -77...",(703) 281-7777,766.688837,tPes69KJaa0NoGc0d9kZdQ,https://s3-media4.fl.yelpcdn.com/bphoto/IhNETD...,False,"{'address1': '118 Branch Rd SE', 'address2': '...",Pazzo Pomodoro,+17032817777,$$,4.0,349,[],https://www.yelp.com/biz/pazzo-pomodoro-vienna...
3,lombardis-vienna,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 38.9012659299093, 'longitude': -7...",(703) 242-5901,316.886278,hcYIJW3EDmH-MfAe8q6mbA,https://s3-media3.fl.yelpcdn.com/bphoto/36XCr3...,False,"{'address1': '115 Maple Ave W', 'address2': ''...",Lombardi's,+17032425901,$$,3.5,38,[],https://www.yelp.com/biz/lombardis-vienna?adju...
4,mod-pizza-vienna-2,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...","{'latitude': 38.8997115224199, 'longitude': -7...",(703) 679-1860,408.434432,Aox_hzHDx6rpBX1AkOKA-Q,https://s3-media2.fl.yelpcdn.com/bphoto/IrnpYw...,False,"{'address1': '168 Maple Ave W', 'address2': No...",MOD Pizza,+17036791860,$,4.0,128,[],https://www.yelp.com/biz/mod-pizza-vienna-2?ad...


## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [19]:
#Your code here
import folium

lat = 38.900782943812246
long = -77.26221084594727

#Create a map of the area
base_map = folium.Map([lat, long], zoom_start=13)
base_map

In [21]:
df['lat'] = df['coordinates'].apply(lambda x: x['latitude'])
df['long'] = df['coordinates'].apply(lambda x: x['longitude'])

In [26]:
#Generate some random locations to add to our map
x = [x for lat in df['lat']]
y = [y for lng in df['long']]
points = list(zip(x, y))
for p in points:
    lat = p[0]
    long = p[1]
    marker = folium.Marker(location=[lat, long])
    marker.add_to(base_map)
base_map

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!